<a href="https://colab.research.google.com/github/DerickW126/Fake-News-Project/blob/main/Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
#import transformers #huggingface transformers library

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
import glob
from google.colab import drive
import os
glob.glob("*.csv.zip")
drive.mount('/content/drive')
%cd /content/drive/MyDrive
os.system("unzip Categorz.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


256

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.79.52.226:8470


REPLICAS:  8


In [ ]:
df = pd.read_json('News_Category_Dataset_v2.json', lines = True)
df.category = df.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)
encoder = LabelEncoder()
df['categoryEncoded'] = encoder.fit_transform(df['category'])
df['headline'] = df['headline'].apply(lambda headline: str(headline).lower())
df['short_description'] = df['short_description'].apply(lambda descr: str(descr).lower())
df['descr_len'] = df['short_description'].apply(lambda x: len(str(x).split()))
df['headline_len'] = df['headline'].apply(lambda x: len(str(x).split()))
df['short_description'] = df['headline'] + df['short_description']

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')
X_train,X_test ,y_train,y_test = train_test_split(df['short_description'], df['categoryEncoded'], random_state = 2020, test_size = 0.3)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [ ]:
Xtrain_encoded = regular_encode(X_train.astype('str'), tokenizer, maxlen=80)

In [ ]:
ytrain_encoded = tf.keras.utils.to_categorical(y_train, num_classes=40,dtype = 'int32')

In [ ]:
Xtest_encoded = regular_encode(X_test.astype('str'), tokenizer, maxlen=80)

In [ ]:

















ytest_encoded = tf.keras.utils.to_categorical(y_test, num_classes=40,dtype = 'int32')

In [ ]:
def build_model(transformer, loss='categorical_crossentropy', max_len=512):
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #adding dropout layer
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    #using a dense layer of 40 neurons as the number of unique categories is 40. 
    out = tf.keras.layers.Dense(40, activation='softmax')(x)
    model = tf.keras.Model(inputs=input_word_ids, outputs=out)
    #using categorical crossentropy as the loss as it is a multi-class classification problem
    model.compile(tf.keras.optimizers.Adam(lr=3e-5), loss=loss, metrics=['accuracy'])
    return model

In [ ]:
#building the model on tpu
with strategy.scope():
    transformer_layer = transformers.TFAutoModel.from_pretrained('bert-large-uncased')
    model = build_model(transformer_layer, max_len=80)
model.summary()

Downloading:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  su

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 80)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  335141888
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             80, 1024),                          
                              pooler_output=(None, 10            
                             24),                                
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [ ]:
#creating the training and testing dataset.
BATCH_SIZE = 32*strategy.num_replicas_in_sync
print(BATCH_SIZE)
AUTO = tf.data.experimental.AUTOTUNE 
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((Xtrain_encoded, ytrain_encoded))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(Xtest_encoded)
    .batch(BATCH_SIZE)
)
n_steps = Xtrain_encoded.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    epochs=10
)

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')
tmp = pd.Series(data=['Black Lives Matter'])
final_tmp = regular_encode(tmp.astype('str'), tokenizer, maxlen=80)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(final_tmp)
    .batch(256)
)
preds = model.predict(test_dataset,verbose = 1)
pred_classes = np.argmax(preds, axis = 1)
encoded_classes=['ARTS',
'ARTS & CULTURE',
'BLACK VOICES',
'BUSINESS',
'COLLEGE',
'COMEDY',
'CRIME',
'CULTURE & ARTS'
,'DIVORCE','EDUCATION'
,'ENTERTAINMENT',
'ENVIRONMENT',
'FIFTY',
'FOOD & DRINK',
'GOOD NEWS',
'GREEN',
'HEALTHY LIVING'
,'HOME & LIVING',
'IMPACT',
'LATINO VOICES',
'MEDIA'
,'MONEY'
,'PARENTING'
,'PARENTS'
,'POLITICS'
,'QUEER VOICES'
,'RELIGION'
,'SCIENCE'
,'SPORTS'
,'STYLE'
,'STYLE & BEAUTY'
,'TASTE'
,'TECH'
,'TRAVEL'
,'WEDDINGS'
,'WEIRD NEWS'
,'WELLNESS'
,'WOMEN'
,'WORLD NEWS'
,'WORLDPOST'
,'COMEDY']
predicted_category = [encoded_classes[x] for x in pred_classes]
print(predicted_category[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'return_attention_masks': False} not recognized.


1/1 [==============================] - 0s 222ms/step
BLACK VOICES


In [ ]:
preds = model.predict(test_dataset,verbose = 1)
#converting the one hot vector output to a linear numpy array.
pred_classes = np.argmax(preds, axis = 1)

236/236 [==============================] - 2197s 9s/step


In [ ]:
encoded_classes = encoder.classes_
#mapping the encoded output to actual categories
predicted_category = [encoded_classes[x] for x in pred_classes]
true_category = [encoded_classes[x] for x in y_test]
encoded_classes = encoder.classes_
result_df = pd.DataFrame({'description':X_test,'true_category':true_category, 'predicted_category':predicted_category})
print(result_df.head())
print(f"Accuracy is {sklearn.metrics.accuracy_score(result_df['true_category'], result_df['predicted_category'])}")

                                             description   true_category  \
75589  huffpost rise morning newsbrief, november 4wel...        POLITICS   
21665  donald trump's lawyer claims president was nev...        POLITICS   
51481  the feminist comic series for fans of 'strange...  ARTS & CULTURE   
20578  obamacare repeal moves ahead with key senate v...        POLITICS   
58445  mount holyoke commencement speaker thanks acti...         COLLEGE   

      predicted_category  
75589           POLITICS  
21665           POLITICS  
51481     ARTS & CULTURE  
20578           POLITICS  
58445            COLLEGE  
Accuracy is 0.6964949548592672


In [ ]:
import pickle
model.save('Category_Model.h5')

In [ ]:
import tensorflow_hub as hub
from tensorflow import keras
import transformers
model = keras.models.load_model('Category_Model.h5',custom_objects={'TFBertModel':transformers.TFBertModel})